### 提问：
Connection、Statement和ResulSet？这三个对象是在方法内部定义的，则这三个对象不是在方法执行完毕就消失了么，为什么还要单独去关闭它们呢？
### 解答：
这个连接是与数据库服务器的一个连接，虽然你的方法结束了，但是这个资源依然存在数据库连接并没有释放
### 提问：
为什么在JDBC对数据库访问结束后，要按先关闭ResultSet，然后关闭PreparedStatement，最后关闭Connection，直接关闭Connection不就行了吗？
### 解答：

感觉上好象是只要把connection给关闭了，系统就能正常运行了。 那在查询或是其它操作中，如果只关闭Connection，不作ResultSet 和 Statement 的关闭的话，对系统性能是否会有影响呢。或者是其它实方面的不良影响。

如果你不使用连接池，那么就没有什么问题，一旦Connection关闭，数据库物理连接就被释放，所有相关Java资源也可以被GC回收了。 但是如果你使用连接池，那么请注意，Connection关闭并不是物理关闭，只是归还连接池，所以PreparedStatement和ResultSet都被持有，并且实际占用相关的数据库的游标资源，在这种情况下，只要长期运行，往往就会报“游标超出数据库允许的最大值”的错误，导致程序无法正 常访问数据库

因为你打开的时候有顺序，

打开时：Connection -> PreparedStatement -> ResultSet

关闭时：ResultSet-> PreparedStatement -> Connection

这个就像 栈，后进先出

```java
    //关闭数据库的资源的顺序最好与使用的顺序相反
    //数据库连接（Connection）非常耗资源，尽量晚创建，尽量早的释放
    //都要加try catch 以防前面关闭出错，后面的就不执行了
    public static void close(ResultSet rs, PreparedStatement pstmt, Connection conn) {
        if (rs != null) {
            try {
                rs.close();
            } catch (Exception e) {
                e.printStackTrace();
            }
        }
        if (pstmt != null) {
            try {
                pstmt.close();
            } catch (Exception e) {
                e.printStackTrace();
            }
        }
        if (conn != null) {
            try {
                conn.close();
            } catch (Exception e) {
                e.printStackTrace();
            }
        }
    }

```

https://www.cnblogs.com/kenx/p/13554581.html

---- 

### Question

I currently learn the JSP & Servlet in java web with tomcat 9.0, 

- I saw [some websites](https://www.tutorialspoint.com/servlets/servlets-life-cycle.htm) say that a servlet will be initialized only once, and the `destroy()` method will be called when the server want to terminate the life of that servlet,

- and a servlet will be used for multiple same type requests, http request for example,

- and database connection resources are important.

So, I think, I should **make the `Connection object` as a member of my Servlet** so that I won't need to establish a new connection every time when there is a new http request, and **close the connection when the servlet is destroyed**. 

like this:

```java
public class LoginServlet extends HttpServlet {
    private Connection conn;

    // init, get mysql Connection
    public TeamBlueServlet() {
        initiate();
    }

    private void initiate() {
        // do something and get connection...
        this.conn = controller.getConnection();
       
    }

    // close connection in this method
    @Override
    public void destroy() {
         // try, catch
         this.conn.close();
    }

    @Override
    public void doGet(HttpServletRequest request, HttpServletResponse response) throws IOException {
        // do some query
        try {
            ResultSet resultSet = ...;
            PreparedStatement statement = ...;
        } catch (SQLException e) {
            ...
        } finally {
            close(statement, resultSet);
        }
    }

    private void close(PreparedStatement sta, ResultSet rs) {
        try {
            if(sta !=null) sta.close();
        } catch (SQLException e) {
            logger.error(e.getMessage());
        }

        try {
            if(rs !=null) rs.close();
        } catch (SQLException e) {
            logger.error(e.getMessage());
        }
    }
}
```

But I also [saw some](https://stackoverflow.com/a/4111621/16317008) say that we should, open connections as late as possible & close connections as soon as possible. 

Therefore I get confused, **I want to know if I need to make the connection and close it with each query or should I keep the connection open**? I know there is something called connection pool probably can solve this problem, but I want to know in this case, which would be better. 

Any help would appreciate, thanks. 

### Answer

如果没有使用connection pool, 就要做到[别人所说的](https://stackoverflow.com/a/4111621/16317008)的:

- Open connections as late as possible
- Close connections as soon as possible


至于为什么, 比如在第一个的http请求来到的时候, servlet被创建, 然后你的数据库连接也创建了, 那之后的每次http 请求都会使用这次建立的连接, 但你并不能保证这个连接没有中断, 如果中断, 你的程序就崩了(因为你只在servlet被创建的时候才建立数据库连接). 

If you're not using a connection pool, making your code much more fragile, then you'll need to create and destroy the connection every time. **You cannot guarantee that the DB server connection will remain usable throughout the life cycle of your servlet**. Note that this is a terrible way to do it and will slow down your application but it will work.

原问题描述: https://stackoverflow.com/q/75817203/16317008

---

另外注意Servlet的life circle,

```java
public class LoginServlet extends HttpServlet {
    private Connection conn;

    // tomcat 并不会对每个请求都 new servlet，看看servlet的life circle
    // 所以这个构造函数只会被调用一次
    // https://stackoverflow.com/a/27765746/16317008
    public TeamBlueServlet() {
        ...
    }

    private void initiate() {
        // do something and get connection...
        this.conn = controller.getConnection();

    }

    // 这个析构函数只会被调用一次, 当servlet对象被摧毁的时候自动被tomcat调用
    @Override
    public void destroy() {
         ...
    }

    @Override
    public void doGet(HttpServletRequest request, HttpServletResponse response) throws IOException {
        // do some query
        try {
            ResultSet resultSet = ...;
            PreparedStatement statement = ...;
        } catch (SQLException e) {
            ...
        } finally {
            close(statement, resultSet);
        }
    }
}
```